In [1]:
import time
import chess
from IPython.display import display, HTML, clear_output
import numpy as np
import pandas as pd
import timeit
import random

# AI Chess Agent Project

## helper functions

### Displays the chess board

In [2]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

### Checks if player agent is white or black

In [3]:
def who(agent):
    return "White" if agent == chess.WHITE else "Black"

### Obtains available moves

In [4]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

### Tallies the white and black players pieces

In [5]:
def count_pieces(board):
    num_pieces = [0,0]
    
    num_pieces[0] += len(board.pieces(chess.PAWN,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.BISHOP, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KING,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.QUEEN,  chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KNIGHT, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.ROOK,   chess.WHITE))

    num_pieces[1] += len(board.pieces(chess.PAWN,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.BISHOP, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KING,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.QUEEN,  chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KNIGHT, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.ROOK,   chess.BLACK))
                             
    return num_pieces

### Plays a single game with two agent players

In [6]:
def play_game(agent1, agent2, visual="svg", pause=0.1):
    """
    agentN1, agent2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = agent1(board)
            else:
                uci = agent2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (False, msg, board)
    game_has_winner = False
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        game_has_winner = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    
    return (game_has_winner, msg, board)

### "Driver" allows for two agent players to play multiple games for a provided number of iterations. Returns a list of scores

In [7]:
def run(agent1, agent2, iterations, agent1_name, agent2_name):
#     df_scoreboard = pd.DataFrame(data={}, columns=['game_result','winner','moves_played', 'remaining_w_pieces','remaining_b_pieces'])    
    scores_list = list()
                                 
    for round_num in range(iterations):
        
        terminal_state = play_game(agent1, agent2, visual="svg", pause=0.001)
#         time = timeit.timeit(play_game(agent1, agent2, visual="svg", pause=0), number=100)/100
        
        game_hase_winner = terminal_state[0]
        msg = terminal_state[1]
        moves_played = len(terminal_state[2].move_stack)
        remaining_w_pieces = count_pieces(terminal_state[2])[0]
        remaining_b_pieces = count_pieces(terminal_state[2])[1]

#         result_list = (game_hase_winner, msg, moves_played, count_pieces(result[2])[0], count_pieces(result[2])[1], result[3])
        result_list = (round_num + 1, iterations, agent1_name, agent2_name, game_hase_winner, msg, moves_played, remaining_w_pieces, remaining_b_pieces)
        scores_list.append(result_list)
        
    return scores_list

### Results Scoreboard

In [8]:
df = pd.DataFrame(columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])

## Random Agent Evaluation

### plays two random agents against eachother 10 times

### Random Agent player

In [9]:
def random_agent(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [10]:
rand_eval_scores = run(random_agent, random_agent, 10, "random_agent", "random_agent")

draw: stalemate


In [11]:
df_rand_eval_scoreboard = pd.DataFrame(data=rand_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])

df_rand_eval_scoreboard.sort_values(by=['moves_played'], inplace=False, ascending=True)

,round_num,iterations,agent1_name,agent2_name,game_has_winner,winner,moves_played,remain_w_pieces,remaining_b_pieces
3,4,10,random_agent,random_agent,True,checkmate: White wins!,181,7,6
6,7,10,random_agent,random_agent,False,checkmate: Black wins!,196,4,7
2,3,10,random_agent,random_agent,True,checkmate: White wins!,209,5,3
5,6,10,random_agent,random_agent,False,checkmate: Black wins!,214,3,3
4,5,10,random_agent,random_agent,False,draw: insufficient material,231,1,2
8,9,10,random_agent,random_agent,False,draw: insufficient material,311,2,1
7,8,10,random_agent,random_agent,False,draw: claim,333,1,2
0,1,10,random_agent,random_agent,False,draw: claim,349,2,1
9,10,10,random_agent,random_agent,False,draw: stalemate,361,3,1
1,2,10,random_agent,random_agent,False,draw: claim,389,3,1


In [12]:
#update results scoreboard
df = df.append(df_rand_eval_scoreboard, ignore_index=True)

### Scoreboard

In [13]:
#10 best games by moves_played ascending
df.sort_values(by=['moves_played'], inplace=False, ascending=True).head(10)

,round_num,iterations,agent1_name,agent2_name,game_has_winner,winner,moves_played,remain_w_pieces,remaining_b_pieces
3,4,10,random_agent,random_agent,True,checkmate: White wins!,181,7,6
6,7,10,random_agent,random_agent,False,checkmate: Black wins!,196,4,7
2,3,10,random_agent,random_agent,True,checkmate: White wins!,209,5,3
5,6,10,random_agent,random_agent,False,checkmate: Black wins!,214,3,3
4,5,10,random_agent,random_agent,False,draw: insufficient material,231,1,2
8,9,10,random_agent,random_agent,False,draw: insufficient material,311,2,1
7,8,10,random_agent,random_agent,False,draw: claim,333,1,2
0,1,10,random_agent,random_agent,False,draw: claim,349,2,1
9,10,10,random_agent,random_agent,False,draw: stalemate,361,3,1
1,2,10,random_agent,random_agent,False,draw: claim,389,3,1


In [ ]:
# #update results scoreboard
# df.append(df_rand_eval_scoreboard, ignore_index=True)

## Naive Agent Evaluation

### Naive evaluation function
Sets the score to 0 and assigns weights to every piece on the board. The weighted sum of all the available pieces on the board is then computed.

The white pieces are assigned positive values while the black ones are assigned negative values of the same magnitude. 

In [ ]:
def naive_eval(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

### Naive Agent
Chooses best score

In [ ]:
def naive_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
# result = play_game(random_agent, naive_agent, visual="svg", pause=0)

In [ ]:
naive_eval_scores = run(naive_agent, random_agent, 10, "naive_agent", "random_agent")

In [ ]:
# df_naive_eval_scoreboard = pd.DataFrame(data=naive_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])
# df_naive_eval_scoreboard

df_naive_eval_scoreboard = pd.DataFrame(data=naive_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])

df_naive_eval_scoreboard.sort_values(by=['moves_played'], inplace=False, ascending=True)


In [ ]:
# #update results scoreboard
# df.append(df_naive_eval_scoreboard, ignore_index=True)

#update results scoreboard
df = df.append(df_naive_eval_scoreboard, ignore_index=True)

### Scoreboard: Top 10 Games With Fewest Moves

In [ ]:
#10 best games by moves_played ascending
df.sort_values(by=['moves_played'], inplace=False, ascending=True).head(10)

### Naive Agent With Improved Evaluation

## Naive Random Heuristic Evaluation
Sets the score to a random value and assigns weights to every piece on the board. The weighted sum of all the available pieces on the board is then computed.

The white pieces are assigned positive values while the black ones are assigned negative values of the same magnitude. 

In [ ]:
def naive_random_heuristic_eval(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

### Naive Agent with Random Heuristic Evaluator
Chooses best score

In [ ]:
def naive_random_heuristic_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_random_heuristic_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
naive_rand_heuristic_eval_scores = run(naive_random_heuristic_agent, random_agent, 10, "naive_random_heuristic_agent", "random_agent")

In [ ]:
# df_naive_rand_heuristic_eval_scoreboard = pd.DataFrame(data=naive_rand_heuristic_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])
# df_naive_rand_heuristic_eval_scoreboard

df_naive_rand_heuristic_eval_scoreboard = pd.DataFrame(data=naive_rand_heuristic_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])
df_naive_rand_heuristic_eval_scoreboard.sort_values(by=['moves_played'], inplace=False, ascending=True)

In [ ]:
#update results scoreboard
df = df.append(df_naive_rand_heuristic_eval_scoreboard , ignore_index=True)

### Scoreboard: Top 10 Games With Fewest Moves

In [ ]:
#10 best games by moves_played ascending
df.sort_values(by=['moves_played'], inplace=False, ascending=True).head(10)

## Minimax

## minimax evaluation
Sets the score to a random value and assigns weights to every piece on the board. The weighted sum of all the available pieces on the board is then computed.

The white pieces are assigned positive values while the black ones are assigned negative values of the same magnitude. 

In [14]:
def minimax_eval(board):
    # moves = list(board.legal_moves)
    # for move in moves:
    #     newboard = board.copy()
    #     # go through board and return a score
    #     move.score = staticAnalysis(newboard, move, board.turn)
    # moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    # return moves[0].uci()
    score = random.random()
    for (piece, value) in [(chess.PAWN, 1), 
                        (chess.BISHOP, 4), 
                        (chess.KING, 0), 
                        (chess.QUEEN, 10), 
                        (chess.KNIGHT, 5),
                        (chess.ROOK, 3)]:
        score += len(board.pieces(piece, True)) * value
        score -= len(board.pieces(piece,False)) * value
        # can also check things about the pieces position here
    return score

In [15]:
def evaluateMoves(board):
    score = 0
    for (piece, value) in [(chess.PAWN, 1), 
                        (chess.BISHOP, 4), 
                        (chess.KING, 0), 
                        (chess.QUEEN, 10), 
                        (chess.KNIGHT, 5),
                        (chess.ROOK, 3)]:
        score += len(board.pieces(piece, True)) * value
        score -= len(board.pieces(piece,False)) * value
        # can also check things about the pieces position here
    score += 100 if board.is_checkmate() else 0
    return score

In [16]:
def maxValue(board, currentAgent, depth,alpha,beta):
    bestMove = -9999

    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        newboard.push_uci(move.uci()) 
        bestMove = max(bestMove,miniMaxDecision(newboard, not currentAgent , depth -1,alpha,beta))
        if bestMove >=beta :
            return bestMove
        alpha = max(alpha, bestMove)
    return bestMove


In [17]:
def minValue(board, currentAgent, depth,alpha,beta):
    bestMove = 9999
    moves = list(board.legal_moves)

    for move in moves:
        newboard = board.copy()
        newboard.push_uci(move.uci()) 
        bestMove = min(bestMove,miniMaxDecision(newboard, not currentAgent, depth -1,alpha,beta))
        if bestMove <= alpha:
            return bestMove
        beta = min(beta, bestMove)
    return bestMove

In [18]:
def miniMaxDecision(board, currentAgent, depth,alpha,beta):
    if depth == 0 :
        return evaluateMoves(board)

    if currentAgent:
        return maxValue(board, currentAgent, depth, alpha, beta)
    else:
        return minValue(board, currentAgent, depth, alpha, beta)

In [19]:
def mini_max_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        newboard.push_uci(move.uci())
        move.score = miniMaxDecision(newboard, False , 3, -10000,10000)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [20]:
minimax_eval_scores = run(mini_max_agent, random_agent, 1, "mini_max_agent", "random_agent")

In [21]:
df_minimax_eval_scoreboard = pd.DataFrame(data=minimax_eval_scores, columns=['round_num', 'iterations', 'agent1_name', 'agent2_name','game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])

df_minimax_eval_scoreboard.sort_values(by=['moves_played'], inplace=False, ascending=True)

,round_num,iterations,agent1_name,agent2_name,game_has_winner,winner,moves_played,remain_w_pieces,remaining_b_pieces
0,1,1,mini_max_agent,random_agent,False,Game interrupted!,6,16,16


In [ ]:
#update results scoreboard
df = df.append(df_minimax_eval_scoreboard, ignore_index=True)

### Scoreboard: Top 10 Games With Fewest Moves

In [ ]:
#10 best games by moves_played ascending
df.sort_values(by=['moves_played'], inplace=False, ascending=True).head(10)

### Games Where Player 1 (white) Wins, Ordered by Moves Played Desc

In [ ]:
d2 = df.loc[df['winner'] == 'checkmate: White wins!']

# df.sort_values(by=['winner','moves_played'], inplace=False, ascending=True).head(10)

In [ ]:
d2.sort_values(by=['moves_played'], inplace=False, ascending=True)

In [ ]:
df

In [ ]:
# import chess.syzygy

In [ ]:
# with chess.syzygy.open_tablebase("data/syzygy/regular") as tablebase:
#     board = chess.Board("8/2K5/4B3/3N4/8/8/4k3/8 b - - 0 1")
#     print(tablebase.probe_wdl(board))

In [ ]:
# board = chess.Board("8/8/8/8/8/8/2Rk4/1K6 b - - 0 1")


In [ ]:
# board

In [ ]:
# curl http://tablebase.lichess.ovh/standard?fen=4k3/6KP/8/8/8/8/7p/8_w_-_-_0_1

In [ ]:
# pip install requests


In [ ]:
# conda install requests

In [ ]:
# import requests

In [ ]:
# response = requests.get("http://tablebase.lichess.ovh/standard?fen=4k3/6KP/8/8/8/8/7p/8_w_-_-_0_1")

In [ ]:
# print(response.json())

In [ ]:
# response

In [ ]:
# response.json()

In [ ]:
# minimax = run(mini_max_agent, random_agent, 10, "mini_max_agent", "random_agent")